# Homework 10: Galaxy Detection Challenge (50 points)

name: XX

email: XX@umich.edu

Kaggle Username: XXXX

This homework assignment took me XX hours in total to complete. (Please help us to gauge the difficulty of the assignment.)

## Collaboration Disclosure

In the cell below, please list *everyone* with whom you discussed any of the homework problems, excluding only the GSIs and the course instructor. 

If you did not discuss the homework with anyone else, write __"I did not discuss this homework with anyone."__

Even if you discuss questions with other, the code you submit must be only yours. All work is checked with the [MOSS plagiarism detector](https://theory.stanford.edu/~aiken/moss/).

I discussed XX with XX.

## Introduction

For this homework, we are hosting a kaggle competition! We hope that you enjoy this opportunity to learn more about machine learning by tackling a real-world dataset and problem. Please take a look at the kaggle page of this competition https://www.kaggle.com/c/umstats507galaxywinter2021. In Kaggle you can find a leaderboard displaying how the class is doing in the public version of the test set results. 

This problem set is designed to be more __open-ended__, you should feel free to experiment with different algorithms or strategies to solve this challenge. We provide a basic implementation using a single layer feedforward neural network, you should feel free to experiment with other architectures and approaches. In the following we provide a non-exhaustive list. 

### General Hints

- For learning more about deep learning, a good resource is Dive into Deep Learning, particularly Chapter 7: http://gluon.ai/chapter_convolutional-modern/index.html

- You do not need to use a GPU to do well on this task. However, bigger models will run faster on a GPU. Google Colab has GPUs available for students to use.

### Neural network architecture

Without changing the type of neural network, here are some changes you can try:

- Changing the number of layers
- Changing the activation function(s) (logistic sigmoid, tanh, relu, leaky relu, ...)
- Modifying the # of hidden layers
- Changing the # units in the hidden layer(s)

Other types of layers that might be useful include: 

- Using dropout layers
- Adding pooling layers
- Adding batch-norm layers
- Adding convolutional layers (since we are using images)

### Training

- Running for more epochs (be careful to avoid over-fitting!)
- Changing the minibatch size
- Early stopping (choosing the model that does the best in a particular epoch even if you train it for longer)
- Using data augmentation methods


### Optimizer 

- Modifying the learning rate, weight_decay, or other parameters of the optimizers. 
- Try other optimizers from the `torch.optim` module. 

### Other approaches

- Using residual layers (aka ResNet)
- Normalizing the images (like in the baseline model below) 
- Other preprocessing steps or [transformations](https://pytorch.org/docs/stable/torchvision/transforms.html) on the images.

__IMPORTANT:__ If you do borrow ideas from external sources to design your algorithm, please reference them explicitly in the **Conclusion** section at the end of this notebook.

## Deliverables

- Submit this jupyter notebook to Canvas as usual.

- A `.csv` file with the output of your algorithm on the `test.pt` images (submitted to Kaggle).

**Hint:** At the end of this notebook, there is a function `counts_to_csv` that you can use to create a `.csv` file that can be uploaded to kaggle for evaluation. 

## Submission to Kaggle

You will need to access Kaggle with the following link: 

https://www.kaggle.com/t/98d81b3ae7434eb6ba68253001803551

and create an account. Please include your Kaggle username at the top of the notebook. 

## Grading

-  Please note you cannot use late days for this assignment due to the University grading schedule.
-  The student with the highest score on kaggle will receive 10 points of extra credit; the second highest scoring student will receive 5 points of extra credit; and the students placing fifth, four, and third will receive 2 points of extra credit.
-  Students with a score above 80% accuracy by April 22 at 9 pm will receive 2 points of extra credit.


## Setup

In [ ]:
%matplotlib inline
import os
import csv
import time
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Data 

We have provided dataset with images of galaxies in `.pt` format (`pytorch` readable) consisting of the following files (included along this notebook in your Canvas download):

- train.pt

- test.pt 

The training file contains __20k__ images of simulated galaxies and testing contains __10k__ galaxies. Each image can contain anything between **0 and 10 galaxies**. Each image has dimensions __20x20__ pixels. Note that the data is grayscale; i.e. there is only one color channel.

The **goal** of your algorithm is to determine the number of galaxies in a given image. You can think of this problem as a classification problem: given an input image we classify it as 1 of the 11 classes (whether the image contains 0, 1, ..., 11 galaxies).

**Note:** You may want to further split up the ```train.pt``` data into a "training" set and a "validation" set, the latter which could be used for tuning hyperparameters, early stopping, etc.

### Overview

The __training set__ contains images and their true counts which can be accessed as follows: 

In [ ]:
train_data = torch.load('train.pt')
train_images = train_data['images']
train_counts = train_data['counts']

print(train_images.shape) # = (10000, 20, 20)
print(train_counts.shape) # = (10000)

In [ ]:
SIDE_LEN = train_images.shape[-1]

To be explicit, let $i$ be an index satisfying $0 < i < 10,000$, then `train_counts[i]` corresponds to the number of galaxies in the 20-by-20 image `train_images[i]`. 

Lets take a look at the first 4 images of galaxies to get an idea of what the data looks like and make sure you can load them correctly.

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(10,10))
axes = axes.flatten() # get a single dimensional array. 
for i in range(9):
    ax = axes[i]
    image = train_images[i].cpu().numpy()
    count = train_counts[i].item()
    ax.imshow(image)
    ax.set_xlabel(f"Number of galaxies: {count}")
plt.tight_layout()

The __testing set__ does not countain the counts of each image, your model will predict the counts and these will be evaluated in kaggle. You can access them like this:

In [ ]:
test_set = torch.load('test.pt')
test_images = test_set['images'] 
assert 'count' not in test_set # does not include counts.
print(test_images.shape) # = (10000, 20, 20)

### Dataset and Dataloader

What follows is a skeleton for a pytorch `Dataset` and corresponding `DataLoaders` that can be used to train and test the model. You can modify this base implementation if you want, but it's not necessary for completing the problem set.

In [ ]:
# You can modify these attributes if you want.
BATCH_SIZE = 32
N_WORKERS = min(4, os.cpu_count())  # number of cpus used to load the data faster.

In [ ]:
from torch.utils.data import Dataset, DataLoader

class GalaxyDataset(Dataset):
    """Custom Dataset for loading Quickdraw images"""

    def __init__(self, filepath, test=False):
        self.data = torch.load(filepath)
        self.images = self.data['images']
        self.test = test
    
    def __len__(self):
        return self.images.shape[0]

    def __getitem__(self, idx):
        
        if self.test: 
            return {'images': self.images[idx]}
        else: 
            return {'images': self.images[idx], 'counts': self.data['counts'][idx]}

In [ ]:
# Define datasets and dataloaders. 
train_dataset = GalaxyDataset('train.pt', test=False)
test_dataset = GalaxyDataset('test.pt', test=True)

## OPTIONAL: Split the training data into "train" and "valid"
train_dataset, valid_dataset = torch.utils.data.random_split(
        train_dataset, [14000, 6000]
    )

In [ ]:
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=N_WORKERS)
valid_loader = DataLoader(dataset=valid_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=N_WORKERS)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False,
                         num_workers=N_WORKERS)

## Utility functions

In [ ]:
def compute_accuracy(model, data_loader):
    """
    Takes in your model and either the train_dataloader and valid_dataloader above. Calculate the accuracy of
    your model in this dataset.

    NOTE: As explained below it assumes your model has a .predict() method. 
    """
    assert hasattr(model, 'predict'), "your model does not have a '.predict()' method."
    correct_pred, n_examples = 0, 0
    with torch.no_grad():
        for batch in data_loader: 
            images = batch['images'].to(DEVICE)
            true_counts = batch['counts'].to(DEVICE)
            pred_counts = model.predict(images)
            n_examples += true_counts.size(0)
            correct_pred += (pred_counts == true_counts).sum()
        return correct_pred.float()/n_examples

In [ ]:
def compute_epoch_loss(model, data_loader):
    """Computes the loss of your model given a train or validation dataloader.
    
    NOTE: Only works for the MLP architecture we have below, if you don't use the MLP architecture 
    then you might need to compute the loss differently. Feel free to modify this function.
    
    """
    curr_loss, n_examples = 0., 0
    with torch.no_grad():
        for batch in data_loader:
            images = batch['images'].to(DEVICE)
            true_counts = batch['counts'].to(DEVICE)
            logits, probs = model(images)
            loss = F.cross_entropy(logits, true_counts, reduction='sum')
            n_examples += true_counts.size(0)
            curr_loss += loss.item()

        curr_loss = curr_loss / n_examples
        return curr_loss

## Your model

### Implementation

Here were provide a baseline model based on a multi-layer perceptron like the one you saw in HW9. You have complete flexibility in the final implementation, including training and how your compute your loss. 

**Note:** The only requirement to be able to test your model is that your class has a method `.predict()` that takes in images of shape `(N, 20, 20)` and returns an array of shape `(N)` containing the prediction for the number of galaxies in each of the images. 

In [ ]:
# the random seed makes sure that the random weight initialization
# in the model is always the same.
# Depending on your architecture you might want to try different seeds since some seeds could correspond
# to a bad initialization.
RANDOM_SEED = 125
torch.manual_seed(RANDOM_SEED)
print("Seed is", RANDOM_SEED)

In [ ]:
class MLP(torch.nn.Module):

    def __init__(self, n_hidden=50, n_classes=7):
        """A standard MLP with 1 hidden layer. 
        
        Arguments: 
            n_hidden: Number of units in the first hidden layer. 
            n_classes: Number of classes. Should be 7, the possible number of galaxies in the image.
        """
        super(MLP, self).__init__()
        n_features=SIDE_LEN**2
        self.linear = torch.nn.Linear(n_features, n_hidden)
        self.linear_out = torch.nn.Linear(n_hidden, n_classes)


    def forward(self, x):
        """Forward function that takes in x, a tensor of shape (N, 20, 20) of galaxy images and 
        returns a prediction.
        
        Returns: 
            logits: Output of the feedfoward neural networs (unnormalized log-probabilities) 
            probs: normalized probabilities. 
        """
        assert len(x.shape) == 3
        assert x.shape[1] == x.shape[2]
        x = x.reshape(-1, SIDE_LEN**2) # flatten because we are doing a linear pass. 
        x = x / x.mean(axis=1).reshape(-1, 1) # I find that normalizing helps. 
        
        out = self.linear(x)
        out = torch.sigmoid(out)
        logits = self.linear_out(out)
        probs = F.softmax(logits, dim=1)
        return logits, probs
    
    def predict(self, x):
        logits, probs = self(x)
        return probs.argmax(axis=1).int()

### Initializiation

You should create your model from the class specified above this cell. You __should__ move your model to the correct device by using the `.to(DEVICE)` method.

In [ ]:
model = MLP(n_classes=11)
model = model.to(DEVICE)

### Optimizer

You can feel free to experiment with different optimizers and learning rates, etc. If you do, this is a good thing to write in the conclusions at the end of this notebook. 

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.02)

## Training

Depending on how much you changed your architecture, your training procedure might change. If you are still using a MLP it's very likely you can keep the same training procedure. 

You can also modify the number of epochs below, as an input argument to `run_training`.

In [ ]:
def run_training(model, train_loader, valid_loader, n_epochs=10):
    start_time = time.time()
    epoch_loss = [] 
    for epoch in range(n_epochs):
        model.train()
        for batch_idx, batch in enumerate(train_loader):
            images = batch['images'].to(DEVICE)
            true_counts = batch['counts'].to(DEVICE)


            # FORWARD AND BACK PROP
            logits, probs = model(images)
            cost = F.cross_entropy(logits, true_counts)
            optimizer.zero_grad()
            cost.backward()

            # UPDATE MODEL PARAMETERS
            optimizer.step()

            # LOGGING
            if not batch_idx % 50:
                print('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f'
                      % (epoch+1, n_epochs, batch_idx,
                         len(train_loader), cost))

        # we evaluate our model after each epoch.
        model.eval()
        cost = compute_epoch_loss(model, train_loader)
        epoch_loss.append(cost)

        train_accuracy = compute_accuracy(model, train_loader)
        valid_accuracy = compute_accuracy(model, valid_loader)

        print('Epoch: %03d/%03d Train Cost: %.4f' % (
                epoch+1, n_epochs, cost))
        print('Train Accuracy: %.3f | Validation Accuracy: %.3f'
              % (train_accuracy, valid_accuracy))
        print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))

    print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))
    return train_accuracy, valid_accuracy, epoch_loss

In [ ]:
N_EPOCHS = 10
train_accuracy, valid_accuracy, epoch_loss = run_training(model, train_loader, valid_loader, n_epochs=N_EPOCHS)

### Loss function

Here is a plot showing your loss as a function of epoch which can be used to see if your model is being optimized correctly. This could also help you decide if e.g. **early stopping** might be helpful. 

In [ ]:
plt.plot(range(len(epoch_loss)), epoch_loss)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()

## Testing 

Once you trained your `model`, you are ready to try it out in the `test_images`. Even though we don't provide the true counts for the test images, you can inspect some by eye if you want. 

**WARNING:** Do not modify the order of `my_counts` otherwise your submission to kaggle will be wrong. 

In [ ]:
my_counts = torch.zeros(0, device=DEVICE) 
model.eval()
for batch in test_loader:
    test_pred = model.predict(batch['images'].to(DEVICE))
    my_counts = torch.cat((my_counts, test_pred), 0)

## Submission (20 points) 

Once you are satisfied with your model and your predictions, you are ready to submit to [kaggle](https://www.kaggle.com/c/galaxy-detection-challenge). We provide the function `counts_to_csv` that automatically takes in your prediction for the test set and creates a CSV file that you can upload to kaggle for your submission. This file is called "kaggle_upload.csv" by default, and will be created in the same directory as your notebook.

In [ ]:
import csv
def counts_to_csv(kaggle_csv, counts):
    assert len(counts.shape) == 1, "Incorrect format for counts."
    assert counts.shape[0] == 10000, "Incorrect format for counts."
    with open(kaggle_csv, 'w', newline='') as fp: 
        writer = csv.writer(fp)
        writer.writerow(['ID', 'Category'])
        for i, c in enumerate(counts):
            writer.writerow([str(i), str(int(c.item()))])

In [ ]:
kaggle_results_csv = "kaggle_upload.csv"
counts_to_csv(kaggle_results_csv, my_counts) # creates a file in your current directory called `UNIQID.csv`.

## Conclusion

In the cell below, please describe the approach you took in designing your algorithm and the choices that went into this design. Your answer should be detailed and will be significantly weighted in determining your final grade. Your answer might include but is not limited to the following points: 

- What type of neural network architecture did you use? (CNN, MLP, ResNet, etc.) How many layers did you use? How many units in each layer? How did you initialize your neural network? What optimizer did you use? 

- What challenges did you encounter as you were doing this problem set? How did you overcome these obstacles?

- Please explain your overall design and the choices behind it as much as possible (this can include your thought process). 

- How many different designs did you try? How did you settle on your final design?

- If you used external resources to get any ideas for your solution, please describe these too.

YOUR ANSWER HERE